# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [0]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.cifar10.load_data()

In [107]:
np.shape(trainX)

(50000, 32, 32, 3)

In [108]:
np.shape(trainY)

(50000, 1)

In [109]:
np.shape(testY)

(10000, 1)

In [0]:
#p=trainY>=5
#p
p=trainY<=4
q=testY<=4

In [111]:
np.shape(trainX[p.reshape(50000,),:,:,:])

(25000, 32, 32, 3)

In [0]:
train_X=(trainX[p.reshape(50000,),:,:,:])
train_Y=trainY[p.reshape(50000,),:]

In [0]:
test_X=(testX[q.reshape(10000,),:,:,:])
test_Y=testY[q.reshape(10000,),:]

In [114]:
print(np.shape(train_X))
print(np.shape(train_Y))
print(np.shape(test_X))
print(np.shape(test_Y))


(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 1)


In [0]:
#p=trainY<=4
p=trainY>=5
q=testY>=5

In [0]:
train_X1=(trainX[p.reshape(50000,),:,:,:])
train_Y1=trainY[p.reshape(50000,),:]

In [0]:
test_X1=(testX[q.reshape(10000,),:,:,:])
test_Y1=testY[q.reshape(10000,),:]

In [118]:
print(np.shape(train_X1))
print(np.shape(train_Y1))
print(np.shape(test_X1))
print(np.shape(test_Y1))

(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [119]:
print(np.unique(train_Y))
print(np.unique(test_Y))
print(np.unique(train_Y1))
print(np.unique(test_Y1))

[0 1 2 3 4]
[0 1 2 3 4]
[5 6 7 8 9]
[5 6 7 8 9]


In [0]:
#Convert labels to one hot encoding
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=5)
#train_Y1 = tf.keras.utils.to_categorical(train_Y1, num_classes=5)

In [0]:
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=5)
#test_Y1 = tf.keras.utils.to_categorical(test_Y1, num_classes=5)

In [122]:
from sklearn.preprocessing import OneHotEncoder
oh=OneHotEncoder()
train_Y1=oh.fit_transform(train_Y1)
test_Y1=oh.transform(test_Y1)
train_Y1.shape

(25000, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
#Clear any existing model in memory
tf.keras.backend.clear_session()

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Reshape data from 2D (32,32,3) to 3D (32, 32, 3)
#model.add(tf.keras.layers.Reshape((32,32,3),input_shape=(32,32,3)))
#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(32,32,3)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))
#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [0]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [125]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

In [126]:
#Train the model
model.fit(train_X,train_Y,          
          validation_data=(test_X,test_Y),
          epochs=40,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/40
25000/25000 [==============================] - 11s 458us/sample - loss: 3.1782 - accuracy: 0.5024 - val_loss: 0.9168 - val_accuracy: 0.6390
Epoch 2/40
25000/25000 [==============================] - 11s 432us/sample - loss: 1.0013 - accuracy: 0.6024 - val_loss: 0.8396 - val_accuracy: 0.6828
Epoch 3/40
25000/25000 [==============================] - 11s 435us/sample - loss: 0.8847 - accuracy: 0.6554 - val_loss: 0.7506 - val_accuracy: 0.7218
Epoch 4/40
25000/25000 [==============================] - 11s 436us/sample - loss: 0.8179 - accuracy: 0.6918 - val_loss: 0.7735 - val_accuracy: 0.7152
Epoch 5/40
25000/25000 [==============================] - 11s 434us/sample - loss: 0.7434 - accuracy: 0.7211 - val_loss: 0.7185 - val_accuracy: 0.7200
Epoch 6/40
25000/25000 [==============================] - 11s 431us/sample - loss: 0.6780 - accuracy: 0.7511 - val_loss: 0.6472 - val_accuracy: 0.7634
Epoch 7/40
25000/25000 [=====================

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
new_model=model

In [128]:
print(len(new_model.layers))

11


In [0]:
#Unfreezing all layers after layer# 158
for layer in new_model.layers[:7]:
    layer.trainable = False 


In [130]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [131]:
new_model.fit(train_X1,train_Y1.todense() ,         
          validation_data=(test_X1,test_Y1.todense()),
          epochs=40,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/40
25000/25000 [==============================] - 11s 438us/sample - loss: 1.7260 - accuracy: 0.2190 - val_loss: 1.4564 - val_accuracy: 0.3488
Epoch 2/40
25000/25000 [==============================] - 11s 426us/sample - loss: 1.4678 - accuracy: 0.3094 - val_loss: 1.2936 - val_accuracy: 0.4632
Epoch 3/40
25000/25000 [==============================] - 11s 425us/sample - loss: 1.4246 - accuracy: 0.3252 - val_loss: 1.2456 - val_accuracy: 0.4568
Epoch 4/40
25000/25000 [==============================] - 11s 428us/sample - loss: 1.4133 - accuracy: 0.3374 - val_loss: 1.2483 - val_accuracy: 0.4420
Epoch 5/40
25000/25000 [==============================] - 11s 427us/sample - loss: 1.3171 - accuracy: 0.3964 - val_loss: 1.1019 - val_accuracy: 0.5290
Epoch 6/40
25000/25000 [==============================] - 11s 435us/sample - loss: 1.1715 - accuracy: 0.4768 - val_loss: 0.9427 - val_accuracy: 0.6414
Epoch 7/40
25000/25000 [=====================

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [136]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [137]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [138]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [140]:
type(twenty_train)

sklearn.utils.Bunch

In [0]:
X_train=twenty_train.data
y_train=twenty_train.target

In [0]:
X_test=twenty_test.data
y_test=twenty_test.target

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [0]:
vect = CountVectorizer(ngram_range=(1, 2), stop_words = 'english', min_df = 2)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
lr = LogisticRegression()

In [0]:
lr.fit(X_train_dtm, y_train)
y_pred_class = lr.predict(X_test_dtm)

In [148]:
# calculate accuracy
print (metrics.accuracy_score(y_test, y_pred_class))

0.9041278295605859
